In [ ]:
from typing import List, Tuple, Dict
from bert_score import score as bert_score
from collections import Counter
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
def is_position_match(pred_pos: Tuple[int, int], gold_pos: Tuple[int, int]) -> bool:
    return pred_pos == gold_pos

def evaluate_missing_prediction(
    gold: Dict[str, List],
    pred: Dict[str, List],
    model_type: str = "roberta-large",
    lang: str = "en",
) -> Dict[str, float]:

    gold_positions = gold["positions"]
    gold_texts = gold["texts"]
    pred_positions = pred["positions"]
    pred_texts = pred["texts"]

    if len(gold_positions) == 0:
        if len(pred_positions) == 0:
            return {
                "precision_pos": 1.0,
                "recall_pos": 1.0,
                "f1_pos": 1.0,
                "redundancy_rate": 0.0,
                "text_score_position_aware": 1.0 
            }
        else:
            return {
                "precision_pos": 0.0,
                "recall_pos": 1.0,
                "f1_pos": 0.0,
                "redundancy_rate": 1.0,
                "text_score_position_aware": 0.0
            }

    matched = []
    unmatched_pred_indices = set(range(len(pred_positions)))
    matched_texts_pred = []
    matched_texts_gold = []

    for i, gpos in enumerate(gold_positions):
        matched_flag = False
        for j, ppos in enumerate(pred_positions):
            if is_position_match(ppos, gpos):
                matched.append((i, j))
                matched_texts_gold.append(gold_texts[i])
                matched_texts_pred.append(pred_texts[j])
                unmatched_pred_indices.discard(j)
                matched_flag = True
                break 
        if not matched_flag:
            matched_texts_gold.append(gold_texts[i])
            matched_texts_pred.append("")

    tp = len(matched)
    fp = len(unmatched_pred_indices)
    fn = len(gold_positions) - tp

    precision_pos = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall_pos = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1_pos = (2 * precision_pos * recall_pos / (precision_pos + recall_pos)) if (precision_pos + recall_pos) > 0 else 0.0
    redundancy_rate = fp / (tp + fp) if (tp + fp) > 0 else 0.0

    P, R, F1 = bert_score(matched_texts_pred, matched_texts_gold, lang=lang, model_type=model_type, verbose=False)
    text_score = float(F1.mean().item()) 

    return {
        "precision_pos": round(precision_pos, 4),
        "recall_pos": round(recall_pos, 4),
        "f1_pos": round(f1_pos, 4),
        "redundancy_rate": round(redundancy_rate, 4),
        "text_score_position_aware": round(text_score, 4),
    }


In [ ]:
from bert_score import score as bert_score
from tqdm import tqdm

def batch_evaluate(
    gold_list: List[Dict[str, List]],
    pred_list: List[Dict[str, List]],
    model_type: str = "roberta-large",
    lang: str = "en",
    show_progress: bool = True,
) -> Dict[str, float]:
    assert len(gold_list) == len(pred_list), "gold/pred inconsistent"

    metrics_sum = {
        "precision_pos": 0.0,
        "recall_pos": 0.0,
        "f1_pos": 0.0,
        "redundancy_rate": 0.0,
    }

    all_gold_texts = []
    all_pred_texts = []
    per_sample_matched_counts = []
    n = len(gold_list)

    iterable = zip(gold_list, pred_list)
    if show_progress:
        iterable = tqdm(iterable, total=n, desc="Evaluating")

    for gold, pred in iterable:
        gold_pos, pred_pos = gold["positions"], pred["positions"]
        gold_texts, pred_texts = gold["texts"], pred["texts"]

        matched = []
        for i, g in enumerate(gold_pos):
            for j, p in enumerate(pred_pos):
                if g == p:
                    matched.append((i, j))
                    break 

        matched_gold_texts = [gold_texts[i] for i, _ in matched]
        matched_pred_texts = [pred_texts[j] for _, j in matched]

        all_gold_texts.extend(matched_gold_texts)
        all_pred_texts.extend(matched_pred_texts)
        per_sample_matched_counts.append(len(matched))

        true_positives = len(matched)
        precision_pos = true_positives / len(pred_pos) if pred_pos else 0.0
        recall_pos = true_positives / len(gold_pos) if gold_pos else 0.0
        f1_pos = (
            2 * precision_pos * recall_pos / (precision_pos + recall_pos)
            if precision_pos + recall_pos > 0 else 0.0
        )
        redundancy_rate = (len(pred_pos) - true_positives) / len(pred_pos) if pred_pos else 0.0

        metrics_sum["precision_pos"] += precision_pos
        metrics_sum["recall_pos"] += recall_pos
        metrics_sum["f1_pos"] += f1_pos
        metrics_sum["redundancy_rate"] += redundancy_rate

    if all_gold_texts:
        _, _, F1 = bert_score(
            all_pred_texts, all_gold_texts,
            lang=lang, model_type=model_type, verbose=False, device="cuda"
        )

        f1_list = F1.tolist()
        idx = 0
        sample_text_scores = []
        for count in per_sample_matched_counts:
            if count == 0:
                sample_text_scores.append(None)
            else:
                sample_f1 = sum(f1_list[idx:idx + count]) / len(gold_list[sample_text_scores.__len__()]["texts"])  # 除以gold中应有的总数
                sample_text_scores.append(sample_f1)
                idx += count
        avg_text_score = round(sum(f for f in sample_text_scores if f is not None) / n, 4)
    else:
        avg_text_score = None

    avg_metrics = {k: round(v / n, 4) for k, v in metrics_sum.items()}
    avg_metrics["text_score_position_aware"] = avg_text_score

    return avg_metrics


In [ ]:
def extract_missing_steps(text, eval_model):
    if eval_model=="ours":
        pattern = re.compile(
            r'Missing Step (\d+)：\s*'
            r'The missing step should be placed between Step (\d+) and Step (\d+)\.\s*'
            r'The missing step is:\s*(.*?)'
            r'(?=\s*Missing Step \d+：|\Z)',
            re.DOTALL
        )
    if eval_model=="72b" or eval_model=="7b":
        pattern = re.compile(
            r'Missing Step (\d+):\s*'
            r'The missing step should be placed between Step (\d+) and Step (\d+)\.\s*'
            r'The missing step is:\s*(.*?)'
            r'(?=\s*Missing Step \d+:|\Z)',
            re.DOTALL
        )
    matches = pattern.findall(text)
    results = []
    for match in matches:
        a, x, y, z = match
        results.append({
            'a': a.strip(),
            'x': x.strip(),
            'y': y.strip(),
            'z': z.strip()
        })
    return results

def filter_results(results):
    filtered_results = []
    for result in results:
        x = int(result['x'])
        y = int(result['y'])
        z = result['z']
        if x + 1 == y and not z.startswith('####'):
            filtered_results.append(result)
    return filtered_results

def sort_results_by_x(results):
    sorted_results = sorted(results, key=lambda result: int(result['x']))
    return sorted_results

def process_text(text, eval_model):
    results = extract_missing_steps(text, eval_model=eval_model)
    filtered_results = filter_results(results)
    sorted_results = sort_results_by_x(filtered_results)
    return sorted_results



In [ ]:
import re
import pickle
with open('results-72b.pkl', 'rb') as f:
    loaded_results = pickle.load(f)
with open('gold.pkl', 'rb') as f:
    gold = pickle.load(f)
predicts = []
for i in range(len(loaded_results)):
    data = process_text(loaded_results[i], eval_model="72b")
    
    predict = {
        "positions": [(int(item["x"]), int(item["y"])) for item in data],
        "texts": [item["z"] for item in data]
    }
    
    predicts.append(predict)
results = batch_evaluate(gold, predicts)
print(results)

In [ ]:
import re
import pickle
with open('results-7b.pkl', 'rb') as f:
    loaded_results = pickle.load(f)
with open('gold.pkl', 'rb') as f:
    gold = pickle.load(f)
predicts = []
for i in range(len(loaded_results)):
    data = process_text(loaded_results[i], eval_model="7b")
    
    predict = {
        "positions": [(int(item["x"]), int(item["y"])) for item in data],
        "texts": [item["z"] for item in data]
    }
    
    predicts.append(predict)
results = batch_evaluate(gold, predicts)
print(results)

In [ ]:
import re
import pickle
with open('results-ours.pkl', 'rb') as f:
    loaded_results = pickle.load(f)
with open('gold.pkl', 'rb') as f:
    gold = pickle.load(f)
predicts = []
for i in range(len(loaded_results)):
    data = process_text(loaded_results[i], eval_model="ours")
    
    predict = {
        "positions": [(int(item["x"]), int(item["y"])) for item in data],
        "texts": [item["z"] for item in data]
    }
    
    predicts.append(predict)
results = batch_evaluate(gold, predicts)
print(results)